# **Group1 BI Project 2**

# 1 - **Data and Packages Import**
## 1.1 - Packages Import

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#import re as re
import seaborn as sns
#import datetime as date

## 1.2- Data Import

In [2]:
bs= pd.read_csv(r'bs140513_032310.csv', encoding="latin8")
bsnet= pd.read_csv (r'bsNET140513_032310.csv')

In [3]:
bs.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [4]:
#bsnet.head()
bsnet.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


## 2 - Data Cleaning

For the analysis we'll consider the **bs DataFrame** because it contains more informations about transactions.

In [5]:
#list of columns where data are between single quote
bs_simple_quote = ["customer", "category", "age", "gender", "zipcodeOri",
                   "merchant", "zipMerchant"]

# Removing quotes
for col in bs_simple_quote:
  bs[col] = bs[col].apply(lambda x: x.replace("'", ""))

bs.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [6]:
# Rename step
bs.rename(columns={"step": "day"}, inplace=True)

In [7]:
bs.isna().sum()

day            0
customer       0
age            0
gender         0
zipcodeOri     0
merchant       0
zipMerchant    0
category       0
amount         0
fraud          0
dtype: int64

In [8]:
bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   day          594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


We realize the type of is not int or age

In [9]:
list(bs["age"].unique())

['4', '2', '3', '5', '1', '6', 'U', '0']

In [10]:
# To put age to int
def to_num(x):
  try:
    x = int(x)
  except ValueError:
    x = np.NaN
  return x

bs["age"] = bs.age.map(lambda x: to_num(x))
bs.age.to_frame().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   age     593465 non-null  float64
dtypes: float64(1)
memory usage: 4.5 MB


In [11]:
bs.describe()

,day,age,amount,fraud
count,594643.000000,593465.000000,594643.000000,594643.000000
mean,94.986827,3.006240,37.890135,0.012108
std,51.053632,1.325342,111.402831,0.109369
min,0.000000,0.000000,0.000000,0.000000
25%,52.000000,2.000000,13.740000,0.000000
50%,97.000000,3.000000,26.900000,0.000000
75%,139.000000,4.000000,42.540000,0.000000
max,179.000000,6.000000,8329.960000,1.000000


**Important**:
    - 1,2108 % of transactions are fraud
    - 37.89 $ by day 
    - On Average, the credit cards have 3 years of use

**Because of age is between 1 and 6, we will make it as categorical variable in order to make some graphiques**

In [12]:

bs["ageCat"] = bs.age.apply(lambda x: "" if np.isnan(x) else str(int(x))+" Years")

In [13]:
bs[bs.age.isna()!=True].head()

,day,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,ageCat
0,0,C1093826151,4.0,M,28007,M348934600,28007,es_transportation,4.55,0,4 Years
1,0,C352968107,2.0,M,28007,M348934600,28007,es_transportation,39.68,0,2 Years
2,0,C2054744914,4.0,F,28007,M1823072687,28007,es_transportation,26.89,0,4 Years
3,0,C1760612790,3.0,M,28007,M348934600,28007,es_transportation,17.25,0,3 Years
4,0,C757503768,5.0,M,28007,M348934600,28007,es_transportation,35.72,0,5 Years


In [14]:
# check full row duplicated 
bs.duplicated().sum()

0

In [15]:
bs.nunique()

day              180
customer        4112
age                7
gender             4
zipcodeOri         1
merchant          50
zipMerchant        1
category          15
amount         23767
fraud              2
ageCat             8
dtype: int64

**Informations***
    - We have data transactions for 180 fays
    - 4 112 customers
    - all the credits cards are delivered at most 6 years ago
    - 50 merchants are concerned
    - there are 15 category of transactions

## Final Data

In [16]:
bsf = bs[bs["fraud"] == 1]
bsnf = bs[bs["fraud"] == 0]

### 




# 3 - Data Visualization

In [17]:
plt.style.use("seaborn-colorblind")

In [18]:
## 3.1 Transactions

In [22]:
fig = plt.figure(figsize=(15,7))

amount_plt = pd.pivot_table(data = bs, values = "amount", index="day", aggfunc = "sum")

sns.lineplot(x=amount_plt.index, y = amount_plt)


ValueError: Shape of passed values is (1, 2), indices imply (180, 2)

<Figure size 1080x504 with 0 Axes>